# Triangle Price Patterns

In [159]:
import pandas as pd
# !pip install yfinance
# import yfinance as yf
# df = yf.download(tickers="TCS.NS",
#                             period="15y",
#                             interval="1d",
#                             auto_adjust=True)
from google.colab import files
uploaded = files.upload()


Saving sample2.csv to sample2 (5).csv


In [160]:
df = pd.read_csv('sample2.csv')
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head(10)

,Gmt time,Open,High,Low,Close,Volume
0,04.05.2003 21:00:00.000,1.12354,1.12354,1.12166,1.12274,95533.0976
1,05.05.2003 01:00:00.000,1.12242,1.12276,1.12067,1.12126,93778.5996
2,05.05.2003 05:00:00.000,1.12139,1.12255,1.12030,1.12113,90924.6992
3,05.05.2003 09:00:00.000,1.12092,1.12331,1.12049,1.12174,91254.6992
4,05.05.2003 13:00:00.000,1.12194,1.12900,1.12130,1.12712,308003.4083
5,05.05.2003 17:00:00.000,1.12718,1.13019,1.12657,1.12804,373668.2930
6,05.05.2003 21:00:00.000,1.12798,1.13004,1.12772,1.12913,94283.7988
7,06.05.2003 01:00:00.000,1.12892,1.12967,1.12743,1.12855,95461.9980
8,06.05.2003 05:00:00.000,1.12856,1.13412,1.12738,1.13381,92809.0996
9,06.05.2003 09:00:00.000,1.13383,1.13662,1.13188,1.13456,90255.7988


In [161]:
# df.describe()
df.shape

(28826, 6)

In [162]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    for i in range(l-n1, l+n2+1):
        if(df1.Low[l]>df1.Low[i]):
            pividlow=0
        if(df1.High[l]<df1.High[i]):
            pividhigh=0
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,3,3), axis=1)

## Pivot Candles Vizualisation

In [163]:
import numpy as np
def pointpos(x):
    if x['pivot']==1:
        return x['Low']-1e-3
    elif x['pivot']==2:
        return x['High']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [164]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
# dfpl = df[28000:28500]
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.add_scatter(x=df.index, y=df['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="pivot")
#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [165]:
import numpy as np
from matplotlib import pyplot
from scipy.stats import linregress

backcandles = 20

candleid = 7310

maxim = np.array([])
minim = np.array([])
xxmin = np.array([])
xxmax = np.array([])

for i in range(candleid-backcandles, candleid+1):
    if df.iloc[i].pivot == 1:
        minim = np.append(minim, df.iloc[i].Low)
        xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
    if df.iloc[i].pivot == 2:
        maxim = np.append(maxim, df.iloc[i].High)
        xxmax = np.append(xxmax, i) # df.iloc[i].name
        
#slmin, intercmin = np.polyfit(xxmin, minim,1) #numpy
#slmax, intercmax = np.polyfit(xxmax, maxim,1)

slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)

print(rmin, rmax)

dfpl = df[candleid-backcandles-10:candleid+backcandles+10]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=4, color="MediumPurple"),
                name="pivot")

#-------------------------------------------------------------------------
# Fitting intercepts to meet highest or lowest candle point in time slice
#adjintercmin = df.low.loc[candleid-backcandles:candleid].min() - slmin*df.low.iloc[candleid-backcandles:candleid].idxmin()
#adjintercmax = df.high.loc[candleid-backcandles:candleid].max() - slmax*df.high.iloc[candleid-backcandles:candleid].idxmax()

xxmin = np.append(xxmin, xxmin[-1]+15)
xxmax = np.append(xxmax, xxmax[-1]+15)
#fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + adjintercmin, mode='lines', name='min slope'))
#fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + adjintercmax, mode='lines', name='max slope'))

fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

0.9225996567254734 1.0


In [166]:
import numpy as np
from matplotlib import pyplot
from scipy.stats import linregress

backcandles= 20
signal = [0] * len(df)
for candleid in range(backcandles, len(df)-3): # important!
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])
    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i].pivot == 1:
            minim = np.append(minim, df.iloc[i].Low)
            xxmin = np.append(xxmin, i) #could be i instead df.iloc[i].name
        if df.iloc[i].pivot == 2:
            maxim = np.append(maxim, df.iloc[i].High)
            xxmax = np.append(xxmax, i) # df.iloc[i].name
    
    if (xxmax.size <3 and xxmin.size <3) or xxmax.size==0 or xxmin.size==0:
        continue
    
    #slmin, intercmin = np.polyfit(xxmin, minim,1) #numpy
    #slmax, intercmax = np.polyfit(xxmax, maxim,1)

    slmin, intercmin, rmin, pmin, semin = linregress(xxmin, minim)
    slmax, intercmax, rmax, pmax, semax = linregress(xxmax, maxim)
        
    buy1 = 0
    sell1 = 0
    buy2 = 0
    sell2 = 0
    if abs(rmax)>=0.9 and abs(rmin)>=0.7 and abs(slmin)<=0.0001 and slmax<-0.0001:
      sell1 = 1
    # else:
    #   sell1 = 0
    if abs(rmax)>=0.9 and abs(rmin)>=0.7 and slmin>=0.0001 and abs(slmax)<=0.00001:
      buy1 = 1
    # else:
    #   buy1 = 0
        
    if abs(rmax)>=0.9 and abs(rmin)>=0.9 and slmin>=0.0001 and slmax<=-0.0001:
        check = 1
        for i in range(candleid, candleid+5):
          if(df['Close'][i]<df['Close'][candleid]):
              check = 0

        if(check==1): sell2 = 1
        # else: sell2 = 0
        
        for i in range(candleid, candleid+5):
          if(df['Close'][i]>df['Close'][candleid]):
            check = 0
        
        if(check==1): buy2 = 1
        # else: buy2 = 0

    if(buy1==1 or buy2==1 and sell1==0 and sell2==0): signal[candleid] = 1
    elif(buy1==0 and buy2==0 and sell1==1 or sell2==1): signal[candleid] = -1
    else: signal[candleid] = 0

        
    # if candleid % 1000 == 0:
    #     print(candleid)

/usr/local/lib/python3.8/dist-packages/scipy/stats/_stats_mstats_common.py:170: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.8/dist-packages/scipy/stats/_stats_mstats_common.py:184: RuntimeWarning:

invalid value encountered in sqrt

/usr/local/lib/python3.8/dist-packages/scipy/stats/_stats_mstats_common.py:187: RuntimeWarning:

invalid value encountered in double_scalars



In [168]:
df['signal'] = signal

,Gmt time,Open,High,Low,Close,Volume,pivot,pointpos,signal
7310,03.01.2008 06:00:00.000,1.47057,1.47328,1.46853,1.47041,184371.9609,1,1.46753,1
16224,13.09.2013 17:00:00.000,1.32885,1.33100,1.32885,1.32952,14944.9799,0,NaN,1
22032,06.06.2017 17:00:00.000,1.12702,1.12843,1.12682,1.12780,25983.7299,2,1.12943,1
22033,06.06.2017 21:00:00.000,1.12781,1.12825,1.12680,1.12727,10589.6897,0,NaN,1
22034,07.06.2017 01:00:00.000,1.12728,1.12740,1.12603,1.12649,14347.7200,0,NaN,1
22035,07.06.2017 05:00:00.000,1.12647,1.12772,1.12463,1.12626,32050.3106,0,NaN,1
25786,04.11.2019 06:00:00.000,1.11680,1.11756,1.11494,1.11652,24225.6804,2,1.11856,1
25787,04.11.2019 10:00:00.000,1.11651,1.11705,1.11523,1.11561,27927.4102,0,NaN,1
25788,04.11.2019 14:00:00.000,1.11563,1.11572,1.11401,1.11415,28176.6801,0,NaN,1
25789,04.11.2019 18:00:00.000,1.11413,1.11430,1.11249,1.11294,14232.3452,0,NaN,1


In [178]:
# New dataset for indices with spurt in their momentum
df_hi = df[df['signal']>0]
df_low = df[df['signal']<0]

not_hi = df[df.index.isin(df_hi.index)].index
not_low = df[df.index.isin(df_low.index)].index
df = df.drop(not_hi)
df = df.drop(not_low)

# set up figure with values not high and not low
# include candlestick with rangeselector
fig = go.Figure(go.Candlestick(x=df.index,
                open=df['Open'], high=df['High'],
                low=df['Low'], close=df['Close']))

# set up trace with extreme highs
fig.add_traces(go.Candlestick(x=df_hi.index,
                open=df_hi['Open'], high=df_hi['High'],
                low=df_hi['Low'], close=df_hi['Close']))

# set up traces with extreme lows
fig.add_traces(go.Candlestick(x=df_low.index,
                open=df_low['Open'], high=df_low['High'],
                low=df_low['Low'], close=df_low['Close']))


color_hi_fill = 'blue'
color_hi_line = 'blue'

color_lo_fill = 'yellow'
color_lo_line = 'yellow'

fig.data[1].increasing.fillcolor = color_hi_fill
fig.data[1].increasing.line.color = color_hi_line
fig.data[1].decreasing.fillcolor = 'rgba(0,0,0,0)'
fig.data[1].decreasing.line.color = 'rgba(0,0,0,0)'

fig.data[2].decreasing.fillcolor = 'rgba(0,0,0,0)'
fig.data[2].decreasing.line.color = 'rgba(0,0,0,0)'
fig.data[2].increasing.fillcolor = color_lo_fill
fig.data[2].increasing.line.color = color_lo_line



fig.show()